In [1]:
# Importação das bibliotecas
from scipy.stats import loguniform, uniform
import numpy as np
import pandas as pd
import os

from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import VGG16
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from sklearn.preprocessing import LabelEncoder, label_binarize
from sklearn.utils import shuffle
from sklearn.model_selection import RandomizedSearchCV

from SwarmPackagePy import gwo
import pyswarms as ps

In [2]:
# Tratamento das imagens
def create_street_data(path, street_types, im_size):
    images, labels = [], []
    streets = [(item, os.path.join(path, item, street)) 
               for item in street_types 
               for street in os.listdir(os.path.join(path, item))]
    streets_df = pd.DataFrame(streets, columns=['street type', 'image'])
    
    for _, row in streets_df.iterrows():
        img = load_img(row['image'], target_size=(im_size, im_size))
        images.append(img_to_array(img))
        labels.append(row['street type'])
    
    return np.array(images, dtype='float32') / 255.0, np.array(labels)

In [3]:
im_size = 224

street_types = ['clean', 'litter', 'recycle']
path = '../Dataset_Menor/'
path_test = '../Dataset_Test/'

In [4]:
train_images, train_labels = create_street_data(path, street_types, im_size)
test_images, test_labels = create_street_data(path_test, street_types, im_size)

streets_count = pd.value_counts(train_labels)
print("Streets in each category:", streets_count)

Streets in each category: clean      50
litter     50
recycle    50
dtype: int64


In [5]:
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

train_x, val_x, train_y, val_y = train_test_split(train_images, train_labels_encoded, test_size=0.15, random_state=415)

print(f"Train shape: {train_x.shape}")
print(f"Validation shape: {val_x.shape}")
print(f"Test shape: {test_images.shape}")

Train shape: (127, 224, 224, 3)
Validation shape: (23, 224, 224, 3)
Test shape: (1081, 224, 224, 3)


In [21]:
# Definição, compilação e treinamento
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(im_size, im_size, 3))

for layer in base_model.layers:
    layer.trainable = False

In [22]:
# Implementação do PSO
def train_top_layer(hyperparameters, train_x, train_y, val_x, val_y, base_model):
    learning_rate, dropout_rate = hyperparameters
    
    top_model = Sequential([
        Flatten(input_shape=base_model.output_shape[1:]),
        Dropout(dropout_rate),
        Dense(3, activation='softmax')
    ])

    model = Model(inputs=base_model.input, outputs=top_model(base_model.output))

    model.compile(optimizer=Adam(learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    history = model.fit(train_x, 
                        train_y, 
                        epochs=5, 
                        validation_data=(val_x, val_y), 
                        verbose=0)
    
    validation_loss, validation_accuracy = model.evaluate(val_x, val_y, verbose=0)
    return validation_loss

In [23]:
def fitness_function(x, train_x, train_y, val_x, val_y, base_model):
    
    n_particles = x.shape[0]
    
    losses = []
    
    for i in range(n_particles):
        hyperparameters = x[i]
        loss = train_top_layer(hyperparameters, train_x, train_y, val_x, val_y, base_model)
        losses.append(loss)
        results.append({'Hiperparâmetros': hyperparameters, 'Perda': loss})
        
    return np.array(losses)

In [24]:
results = []

bounds = [(0.0001, 0.1), (0.0, 0.5)]
options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}

optimizer = ps.single.GlobalBestPSO(n_particles=5, 
                                    dimensions=2, 
                                    options=options, 
                                    bounds=bounds)

cost, best_pos = optimizer.optimize(fitness_function, 
                                    iters=5, 
                                    train_x=train_x, 
                                    train_y=train_y, 
                                    val_x=val_x, 
                                    val_y=val_y,
                                    base_model=base_model)

best_learning_rate, best_dropout_rate = best_pos

2023-12-12 00:14:49,088 - pyswarms.single.global_best - INFO - Optimize for 5 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|5/5, best_cost=0.591
2023-12-12 00:58:58,086 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.5905982851982117, best pos: [1.66970166e-04 4.01440269e-01]


In [25]:
# Definição, compilação e treinamento
base_model_output = base_model.output
top_layers_output = Flatten()(base_model_output)
top_layers_output = Dropout(best_dropout_rate)(top_layers_output)
top_layers_output = Dense(3, activation='softmax')(top_layers_output)

final_model = Model(inputs=base_model.input, outputs=top_layers_output)

final_model.compile(optimizer=Adam(best_learning_rate),
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])

history = final_model.fit(train_x, 
                          train_y, 
                          epochs=5, 
                          validation_data=(val_x, val_y))

Epoch 1/5
4/4 [==============================] - 23s 6s/step - loss: 1.2365 - accuracy: 0.4016 - val_loss: 1.0523 - val_accuracy: 0.2609
Epoch 2/5
4/4 [==============================] - 20s 5s/step - loss: 1.0483 - accuracy: 0.4961 - val_loss: 0.9145 - val_accuracy: 0.6087
Epoch 3/5
4/4 [==============================] - 20s 5s/step - loss: 0.7623 - accuracy: 0.6614 - val_loss: 0.8000 - val_accuracy: 0.6087
Epoch 4/5
4/4 [==============================] - 21s 6s/step - loss: 0.6032 - accuracy: 0.7638 - val_loss: 0.7118 - val_accuracy: 0.7826
Epoch 5/5
4/4 [==============================] - 22s 6s/step - loss: 0.4910 - accuracy: 0.8110 - val_loss: 0.7047 - val_accuracy: 0.7826


In [26]:
# Avaliação e análise
best_results = []
best_results.append({'Melhor taxa de aprendizado': best_learning_rate, 'Melhor taxa de dropout': best_dropout_rate})

print(f"Melhor taxa de aprendizado: {best_learning_rate}")
print(f"Melhor taxa de dropout: {best_dropout_rate}")

test_loss, test_accuracy = final_model.evaluate(test_images, test_labels_encoded, verbose=1)
print(f"\nAcurácia no teste: {test_accuracy*100:.2f}%\n")

y_pred_test = final_model.predict(test_images)
y_pred_test_classes = np.argmax(y_pred_test, axis=1)

confusion_mtx = confusion_matrix(test_labels_encoded, y_pred_test_classes)
print("Matriz de Confusão:")
print(confusion_mtx)

class_report = classification_report(test_labels_encoded, y_pred_test_classes, target_names=label_encoder.classes_)
print("\nRelatório de Classificação:")
print(class_report)

y_true_binarized = to_categorical(test_labels_encoded)

for i in range(y_true_binarized.shape[1]):
    auc_score = roc_auc_score(y_true_binarized[:, i], y_pred_test[:, i])
    print(f'AUC para a classe {label_encoder.classes_[i]}: {auc_score:.2f}')

Melhor taxa de aprendizado: 0.00016697016593511955
Melhor taxa de dropout: 0.40144026943574496
34/34 [==============================] - 153s 4s/step - loss: 0.5769 - accuracy: 0.7946

Acurácia no teste: 79.46%

34/34 [==============================] - 154s 5s/step
Matriz de Confusão:
[[336  24   5]
 [101 234   6]
 [ 79   7 289]]

Relatório de Classificação:
              precision    recall  f1-score   support

       clean       0.65      0.92      0.76       365
      litter       0.88      0.69      0.77       341
     recycle       0.96      0.77      0.86       375

    accuracy                           0.79      1081
   macro avg       0.83      0.79      0.80      1081
weighted avg       0.83      0.79      0.80      1081

AUC para a classe clean: 0.93
AUC para a classe litter: 0.95
AUC para a classe recycle: 0.98


In [28]:
# Armazenamento do Excel e Modelo
shapes_data = {
    'Conjunto': ['Treino', 'Validação', 'Teste'],
    'Formato': [train_x.shape, val_x.shape, test_images.shape]
}
df_shapes = pd.DataFrame(shapes_data)

df_results = pd.DataFrame(results)

df_best_results = pd.DataFrame(best_results)

df_confusion_mtx = pd.DataFrame(confusion_mtx, index=label_encoder.classes_, columns=label_encoder.classes_)

report_data = classification_report(test_labels_encoded, y_pred_test_classes, target_names=label_encoder.classes_, output_dict=True)
df_class_report = pd.DataFrame(report_data).transpose()

auc_scores = {label_encoder.classes_[i]: roc_auc_score(y_true_binarized[:, i], y_pred_test[:, i]) for i in range(y_true_binarized.shape[1])}
df_auc_scores = pd.DataFrame(list(auc_scores.items()), columns=['Classe', 'AUC'])

with pd.ExcelWriter('IC_Project_Fase3_PSO_Excel.xlsx') as writer:
    df_shapes.to_excel(writer, sheet_name='Formas dos Conjuntos', index=False)
    df_results.to_excel(writer, sheet_name='Resultados Otimizacao', index=False)
    df_best_results.to_excel(writer, sheet_name='Melhor Resultado', index=False)
    df_confusion_mtx.to_excel(writer, sheet_name='Matriz de Confusão')
    df_class_report.to_excel(writer, sheet_name='Relatório de Classificação')
    df_auc_scores.to_excel(writer, sheet_name='AUC por Classe', index=False)

print("Resultados exportados para o arquivo 'IC_Project_Fase3_PSO_Excel.xlsx'")

Resultados exportados para o arquivo 'IC_Project_Fase3_PSO_Excel.xlsx'


In [29]:
model_save_path = 'IC_Project_Fase3_PSO_SaveModel.h5'

final_model.save(model_save_path)
print(f"Modelo salvo com sucesso em: {model_save_path}")

Modelo salvo com sucesso em: IC_Project_Fase3_PSO_SaveModel.h5
